In [34]:
import os

os.environ["NEO4J_URI"] = "neo4j+s://e4aa044e.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "NmObplaapQCdBkswnB2aQJPmlunmvH7LU4xabbP489Y"

In [35]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()

ValueError: Cannot resolve address e4aa044e.databases.neo4j.io:7687

In [32]:
from langchain.document_loaders import WikipediaLoader

# Read the wikipedia article
raw_documents = WikipediaLoader(query="Elizabeth I").load()

/opt/anaconda3/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /opt/anaconda3/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [33]:
from langchain.text_splitter import TokenTextSplitter

# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size = 512, chunk_overlap = 24)
documents = text_splitter.split_documents(raw_documents[:3])

In [18]:
from langchain_ollama.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer

llm = ChatOllama(model = "llama3.1:8b-instruct-fp16")
llm_transformer = LLMGraphTransformer(llm = llm)

# Extract graph data
graph_documents = llm_transformer.convert_to_graph_documents(documents)

# Store to neo4j
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel = True,
    include_source = True
)

In [22]:
from langchain_community.vectorstores import Neo4jVector
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model = "llama3.1:8b-instruct-fp16")

vector_index = Neo4jVector.from_existing_graph( # Same as vector_store
    embeddings,
    search_type = "hybrid",
    node_label = "Document",
    text_node_properties = ["text"],
    embedding_node_property = "embedding"
)

In [27]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate

# Extract entities from the text using Pydantic Schema
class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
    ...,
    description="All the person, organization, or business entities that appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following"
            "input: {question}",
        ),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

In [28]:
# Testing it out
entity_chain.invoke({"question": "Where was Amelia Earhart born?"})

Entities(names=['Amelia Earhart'])

In [29]:
graph.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text
    search. It processes the input string by splitting it into words and 
    appending a similarity threshold (~2 changed characters) to each
    word, then combines them using the AND operator. Useful for mapping
    entities from user questions to database values, and allows for some 
    misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

ValueError: Cannot resolve address e4aa044e.databases.neo4j.io:7687

In [ ]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, 
            {limit:2})
            YIELD node,score
            CALL {
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS 
              output
              UNION
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS 
              output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

ADD GITIGNORE TO REMOVE